# MDX-Net Colab
<div style="display:flex; align-items:center; font-size: 16px;">
  <img src="https://github.githubassets.com/pinned-octocat.svg" alt="icon1" style="margin-right:10px; height: 20px;" width="1.5%">
  <span>Trained models provided in this notebook are from <a href="https://github.com/Anjok07">UVR-GUI</a>.</span>
</div>
<div style="display:flex; align-items:center; font-size: 16px;">
  <img src="https://github.com/Anjok07/ultimatevocalremovergui/raw/master/gui_data/img/GUI-Icon.ico" alt="icon2" style="margin-right:10px; height: 20px;margin-top:10px" width="1.5%">
  <span>OFFICIAL UVR GITHUB PAGE: <a href="https://github.com/Anjok07/ultimatevocalremovergui">here</a>.</span>
</div>
<div style="display:flex; align-items:center; font-size: 16px;">
  <img src="https://avatars.githubusercontent.com/u/24620594" alt="icon3" style="margin-right:10px; height: 20px;" width="1.5%">
  <span>OFFICIAL CLI Version: <a href="https://github.com/tsurumeso/vocal-remover">here</a>.</span>
</div>
<div style="display:flex; align-items:center; font-size: 16px;">
  <img src="https://icons.getbootstrap.com/assets/icons/discord.svg" alt="icon4" style="margin-right:10px; height: 20px;" width="1.5%">
  <span>Join our <a href="https://cutt.ly/0TcDjmo">Discord server</a>!</span>
</div>
<sup><br>Ultimate Vocal Remover (unofficial)</sup>
<sup><br>MDX-Net by <a href="https://github.com/kuielab">kuielab</a> and adapted for Colaboratory by <a href="https://www.youtube.com/channel/UC0NiSV1jLMH-9E09wiDVFYw">AudioHacker</a>.</sup>

<sup><br>Your support means a lot to me. If you enjoy my work, please consider buying me a ko-fi:<br></sup>
[![ko-fi](https://ko-fi.com/img/githubbutton_sm.svg)](https://ko-fi.com/X8X6M8FR0)

In [ ]:
import json
import os
import os.path
import gc
import psutil
import requests
import subprocess
import glob
import time
import logging
import sys
from bs4 import BeautifulSoup
from google.colab import drive, files, output
from IPython.display import Audio, display

if "first_cell_ran" in locals():
    print("You've ran this cell for this session. No need to run it again.\nif you think something went wrong or you want to change mounting path, restart the runtime.")
else:
    print('Setting up... please wait around 1-2 minute(s).')

    branch = "https://github.com/NaJeongMo/Colab-for-MDX_B"

    model_params = "https://raw.githubusercontent.com/TRvlvr/application_data/main/mdx_model_data/model_data.json"
    _Models = "https://github.com/TRvlvr/model_repo/releases/download/all_public_uvr_models/"
    # _models = "https://pastebin.com/raw/jBzYB8vz"
    _models = "https://raw.githubusercontent.com/TRvlvr/application_data/main/filelists/download_checks.json"
    stem_naming = "https://pastebin.com/raw/mpH4hRcF"
    arl_check_endpoint = 'https://dz.doubledouble.top/check' # param: arl?=<>

    file_folder = "Colab-for-MDX_B"

    model_ids = requests.get(_models).json()
    model_ids = model_ids["mdx_download_list"].values()

    model_params = requests.get(model_params).json()
    stem_naming = requests.get(stem_naming).json()

    os.makedirs("tmp_models", exist_ok=True)

    # @markdown If you don't wish to mount google drive, uncheck this box.
    MountDrive = True  # @param{type:"boolean"}
    # @markdown The path for the drive to be mounted: Please be cautious when modifying this as it can cause issues if not done properly.
    mounting_path = "/content/drive/MyDrive"  # @param ["snippets:","/content/drive/MyDrive","/content/drive/Shareddrives/<your shared drive name>", "/content/drive/Shareddrives/Shared Drive"]{allow-input: true}
    # @markdown Force update and disregard local changes: discards all local modifications in your repository, effectively replacing all files with the versions from the original commit.
    force_update = False  # @param{type:"boolean"}
    # @markdown Auto Update (does not discard your changes)
    auto_update = True  # @param{type:"boolean"}


    reqs_apt = []  # !sudo apt-get install
    reqs_pip = ["librosa>=0.6.3,<0.9", "onnxruntime_gpu", "deemix", "yt_dlp"]  # pip3 install

    class hide_opt:  # hide outputs
        def __enter__(self):
            self._original_stdout = sys.stdout
            sys.stdout = open(os.devnull, "w")

        def __exit__(self, exc_type, exc_val, exc_tb):
            sys.stdout.close()
            sys.stdout = self._original_stdout

    def get_size(bytes, suffix="B"):  # read ram
        global svmem
        factor = 1024
        for unit in ["", "K", "M", "G", "T", "P"]:
            if bytes < factor:
                return f"{bytes:.2f}{unit}{suffix}"
            bytes /= factor
        svmem = psutil.virtual_memory()


    print('installing requirements...',end=' ')
    with hide_opt():
        for x in reqs_apt:
            subprocess.run(["sudo", "apt-get", "install", x])
        for x in reqs_pip:
            subprocess.run(["python3", "-m", "pip", "install", x])
    print('done')

    def install_or_mount_drive():
        print(
            "Please log in to your account by following the prompts in the pop-up tab.\nThis step is necessary to install the files to your Google Drive.\nIf you have any concerns about the safety of this notebook, you can choose not to mount your drive by unchecking the \"MountDrive\" checkbox."
        )
        drive.mount("/content/drive", force_remount=True)
        os.chdir(mounting_path)
        # check if previous installation is done
        if os.path.exists(os.path.join(mounting_path, file_folder)):
            # update checking
            os.chdir(file_folder)

            if force_update:
                print('Force updating...')

                commands = [
                    ["git", "pull"],
                    ["git", "checkout", "--", "."],
                ]

                for cmd in commands:
                    subprocess.run(cmd)

            elif auto_update:
                print('Checking for updates...')
                commands = [
                    ["git", "pull"],
                ]

                for cmd in commands:
                    subprocess.run(cmd)
        else:
            subprocess.run(["git", "clone", "https://github.com/NaJeongMo/Colab-for-MDX_B.git"])
            os.chdir(file_folder)

    def use_uvr_without_saving():
        global mounting_path
        print("Notice: files won't be saved to personal drive.")
        print(f"Downloading {file_folder}...", end=" ")
        mounting_path = "/content"
        with hide_opt():
            os.chdir(mounting_path)
            subprocess.run(["git", "clone", "https://github.com/NaJeongMo/Colab-for-MDX_B.git"])
            os.chdir(file_folder)

    if MountDrive:
        install_or_mount_drive()
    else:
        use_uvr_without_saving()
    print("done!")
    if not os.path.exists("tracks"):
        os.mkdir("tracks")

    print('Importing required libraries...',end=' ')

    import os
    import mdx
    import librosa
    import torch
    import soundfile as sf
    import numpy as np
    import yt_dlp

    from deezer import Deezer
    from deezer import TrackFormats
    import deemix
    from deemix.settings import load as loadSettings
    from deemix.downloader import Downloader
    from deemix import generateDownloadObject

    logger = logging.getLogger("yt_dlp")
    logger.setLevel(logging.ERROR)

    def id_to_ptm(mkey):
        if mkey in model_ids:
            mpath = f"/content/tmp_models/{mkey}"
            if not os.path.exists(f'/content/tmp_models/{mkey}'):
                print('Downloading model...',end=' ')
                subprocess.run(
                    ["wget", _Models+mkey, "-O", mpath]
                )
                print(f'saved to {mpath}')
                # get_ipython().system(f'gdown {model_id} -O /content/tmp_models/{mkey}')
                return mpath
            else:
                return mpath
        else:
            mpath = f'models/{mkey}'
            return mpath

    def prepare_mdx(custom_param=False, dim_f=None, dim_t=None, n_fft=None, stem_name=None, compensation=None):
        device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
        if custom_param:
            assert not (dim_f is None or dim_t is None or n_fft is None or compensation is None), 'Custom parameter selected, but incomplete parameters are provided.'
            mdx_model = mdx.MDX_Model(
                device,
                dim_f = dim_f,
                dim_t = dim_t,
                n_fft = n_fft,
                stem_name=stem_name,
                compensation=compensation
            )
        else:
            model_hash = mdx.MDX.get_hash(onnx)
            if model_hash in model_params:
                mp = model_params.get(model_hash)
                mdx_model = mdx.MDX_Model(
                    device,
                    dim_f = mp["mdx_dim_f_set"],
                    dim_t = 2**mp["mdx_dim_t_set"],
                    n_fft = mp["mdx_n_fft_scale_set"],
                    stem_name=mp["primary_stem"],
                    compensation=compensation if not custom_param and compensation is not None else mp["compensate"]
                )
        return mdx_model

    def run_mdx(onnx, mdx_model,filename,diff=False,suffix=None,diff_suffix=None, denoise=False, m_threads=1):
        mdx_sess = mdx.MDX(onnx,mdx_model)
        print(f"Processing: {filename}")
        wave, sr = librosa.load(filename,mono=False, sr=44100)
        # normalizing input wave gives better output
        peak = max(np.max(wave), abs(np.min(wave)))
        wave /= peak
        if denoise:
            wave_processed = -(mdx_sess.process_wave(-wave, m_threads)) + (mdx_sess.process_wave(wave, m_threads))
            wave_processed *= 0.5
        else:
            wave_processed = mdx_sess.process_wave(wave, m_threads)
        # return to previous peak
        wave_processed *= peak

        stem_name = mdx_model.stem_name if suffix is None else suffix # use suffix if provided
        save_path = f"{os.path.basename(os.path.splitext(filename)[0])}_{stem_name}.wav"
        save_path = os.path.join(
                'separated',
                save_path
            )
        sf.write(
            save_path,
            wave_processed.T,
            sr
        )

        print(f'done, saved to: {save_path}')

        if diff:
            diff_stem_name = stem_naming.get(stem_name) if diff_suffix is None else diff_suffix # use suffix if provided
            stem_name = f"{stem_name}_diff" if diff_stem_name is None else diff_stem_name
            save_path = f"{os.path.basename(os.path.splitext(filename)[0])}_{stem_name}.wav"
            save_path = os.path.join(
                    'separated',
                    save_path
                )
            sf.write(
                save_path,
                (-wave_processed.T*mdx_model.compensation)+wave.T,
                sr
            )
            print(f'invert done, saved to: {save_path}')
        del mdx_sess, wave_processed, wave
        gc.collect()

    def is_valid_url(url):
        import re
        regex = re.compile(
            r'^https?://'
            r'(?:(?:[A-Z0-9](?:[A-Z0-9-]{0,61}[A-Z0-9])?\.)+[A-Z]{2,6}\.?|'
            r'localhost|'
            r'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3})'
            r'(?::\d+)?'
            r'(?:/?|[/?]\S+)$', re.IGNORECASE)
        return url is not None and regex.search(url)

    def download_deezer(link, arl, fmt='FLAC'):
        match fmt:
            case 'FLAC':
                bitrate = TrackFormats.FLAC
            case 'MP3_320':
                bitrate = TrackFormats.MP3_320
            case 'MP3_128':
                bitrate = TrackFormats.MP3_128
            case _:
                bitrate = TrackFormats.MP3_128

        dz = Deezer()
        settings = loadSettings('dz_config')
        settings['downloadLocation'] = './tracks'
        if not dz.login_via_arl(arl.strip()):
            raise Exception('Error while logging in with provided ARL.')
        downloadObject = generateDownloadObject(dz, link, bitrate)
        print(f'Downloading {downloadObject.type}: "{downloadObject.title}" by {downloadObject.artist}...',end=' ',flush=True)
        Downloader(dz, downloadObject, settings).start()
        print(f'done.')

        path_to_audio = []
        for file in downloadObject.files:
            path_to_audio.append(file["path"])

        return path_to_audio

    def download_link(url):
        ydl_opts = {
            'format': 'bestvideo+bestaudio/best',
            'outtmpl': '%(title)s.%(ext)s',
            'nocheckcertificate': True,
            'ignoreerrors': True,
            'no_warnings': True,
            'extractaudio': True,
        }
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            result = ydl.extract_info(url, download=True)
            download_path = ydl.prepare_filename(result)
        return download_path

    print('finished setting up!')
    first_cell_ran = True

In [ ]:
if 'first_cell_ran' in locals():
    os.chdir(mounting_path + '/' + file_folder + '/')
    #parameter markdowns-----------------
    #@markdown ### Input files
    #@markdown track filename: Upload your songs to the "tracks" folder. You may provide multiple links/files by spliting them with ;
    filename = "https://deezer.com/album/281108671" #@param {type:"string"}
    #@markdown onnx model (if you have your own model, upload it in models folder)
    onnx = "UVR-MDX-NET-Inst_HQ_3.onnx" #@param ["Kim_Inst.onnx", "Kim_Vocal_1.onnx", "Kim_Vocal_2.onnx", "kuielab_a_bass.onnx", "kuielab_a_drums.onnx", "kuielab_a_other.onnx", "kuielab_a_vocals.onnx", "kuielab_b_bass.onnx", "kuielab_b_drums.onnx", "kuielab_b_other.onnx", "kuielab_b_vocals.onnx", "Reverb_HQ_By_FoxJoy.onnx", "UVR-MDX-NET-Inst_1.onnx", "UVR-MDX-NET-Inst_2.onnx", "UVR-MDX-NET-Inst_3.onnx", "UVR-MDX-NET-Inst_HQ_1.onnx", "UVR-MDX-NET-Inst_HQ_2.onnx", "UVR-MDX-NET-Inst_Main.onnx", "UVR_MDXNET_1_9703.onnx", "UVR_MDXNET_2_9682.onnx", "UVR_MDXNET_3_9662.onnx", "UVR_MDXNET_9482.onnx", "UVR_MDXNET_KARA.onnx", "UVR_MDXNET_KARA_2.onnx", "UVR_MDXNET_Main.onnx", "UVR-MDX-NET-Inst_HQ_3.onnx", "UVR-MDX-NET-Voc_FT.onnx"]{allow-input: true}
    #@markdown process all: processes all tracks inside tracks/ folder instead. (filename will be ignored!)
    process_all = False  # @param{type:"boolean"}


    #@markdown ### Settings
    #@markdown invert: get difference between input and output (e.g get Instrumental out of Vocals)
    invert = True  # @param{type:"boolean"}
    #@markdown denoise: get rid of MDX noise. (This processes input track twice)
    denoise = True  # @param{type:"boolean"}
    #@markdown m_threads: like batch size, processes input wave in n threads. (beneficial for CPU)
    m_threads = 2 #@param {type:"slider", min:1, max:8, step:1}

    #@markdown ### Custom model parameters (Only use this if you're using new/unofficial/custom models)
    #@markdown Use custom model parameters. (Default: unchecked, or auto)
    use_custom_parameter = False  # @param{type:"boolean"}
    #@markdown Output file suffix (usually the stem name e.g Vocals)
    suffix = "Vocals_custom" #@param ["Vocals", "Drums", "Bass", "Other"]{allow-input: true}
    suffix_invert = "Instrumental_custom" #@param ["Instrumental", "Drumless", "Bassless", "Instruments"]{allow-input: true}
    #@markdown Model parameters
    dim_f = 3072 #@param {type: "integer"}
    dim_t = 256 #@param {type: "integer"}
    n_fft = 6144 #@param {type: "integer"}
    #@markdown use custom compensation: only if you have your own compensation value for your model. this still apply even if you don't have use_custom_parameter checked (Default: unchecked, or auto)
    use_custom_compensation = False  # @param{type:"boolean"}
    compensation = 1.000 #@param {type: "number"}

    #@markdown ### Extras
    #@markdown Deezer arl: paste your ARL here for deezer tracks directly!
    arl = "" #@param {type:"string"}
    #@markdown Track format: select track quality/format
    track_format = "FLAC" #@param ["FLAC","MP3_320","MP3_128"]
    #@markdown Print settings being used in the run
    print_settings = True  # @param{type:"boolean"}



    onnx = id_to_ptm(onnx)
    compensation = compensation if use_custom_compensation or use_custom_parameter else None
    mdx_model = prepare_mdx(use_custom_parameter, dim_f, dim_t, n_fft, compensation=compensation)

    filename_split = filename.split(';')

    usable_files = []

    if not process_all:
        for fn in filename_split:
            fn = fn.strip()
            if is_valid_url(fn):
                dm, ltype, lid = deemix.parseLink(fn)
                if ltype and lid:
                    usable_files += download_deezer(fn, arl, track_format)
                else:
                    print('downloading link...',end=' ')
                    usable_files+=[download_link(fn)]
                    print('done')
            else:
                usable_files.append(os.path.join('tracks',fn))
    else:
        for fn in glob.glob('tracks/*'):
            usable_files.append(fn)
    for filename in usable_files:
        suffix_naming = suffix if use_custom_parameter else None
        diff_suffix_naming = suffix_invert if use_custom_parameter else None
        run_mdx(onnx, mdx_model, filename, diff=invert,suffix=suffix_naming,diff_suffix=diff_suffix_naming,denoise=denoise)

    if print_settings:
        print()
        print('[MDX-Net_Colab settings used]')
        print(f'Model used: {onnx}')
        print(f'Model MD5: {mdx.MDX.get_hash(onnx)}')
        print(f'Using de-noise: {denoise}')
        print(f'Model parameters:')
        print(f'    -dim_f: {mdx_model.dim_f}')
        print(f'    -dim_t: {mdx_model.dim_t}')
        print(f'    -n_fft: {mdx_model.n_fft}')
        print(f'    -compensation: {mdx_model.compensation}')
        print()
        print('[Input file]')
        print('filename(s): ')
        for filename in usable_files:
            print(f'    -{filename}')

    del mdx_model

# Guide

This tutorial guide will walk you through the steps to use the features of this Colab notebook.

## Mount Drive

To mount your Google Drive, follow these steps:

1. Check the box next to "MountDrive" if you want to mount Google Drive.
2. Modify the "mounting_path" if you want to specify a different path for the drive to be mounted. **Note:** Be cautious when modifying this path as it can cause issues if not done properly.
3. Check the box next to "Force update and disregard local changes" if you want to discard all local modifications in your repository and replace the files with the versions from the original commit.
4. Check the box next to "Auto Update" if you want to automatically update without discarding your changes. Leave it unchecked if you want to manually update.

## Input Files

To upload your songs, follow these steps:

1. Specify the "track filename" for your songs. You can provide multiple links or files by separating them with a semicolon (;).
2. Upload your songs to the "tracks" folder.

## ONNX Model

If you have your own ONNX model, follow these steps:

1. Upload your model to the "models" folder.
2. Specify the "onnx" filename for your model.

## Processing

To process your tracks, follow these steps:

1. If you want to process all tracks inside the "tracks" folder, check the box next to "process_all" and ignore the "filename" field.
2. Specify any additional settings you want:
   - Check the box next to "invert" to get the difference between input and output (e.g., get Instrumental out of Vocals).
   - Check the box next to "denoise" to get rid of MDX noise. This processes the input track twice.
   - Specify custom model parameters only if you're using new/unofficial/custom models. Use the "use_custom_parameter" checkbox to enable this feature.
   - Specify the output file suffix, which is usually the stem name (e.g., Vocals). Use the "suffix" field to specify the suffix for normal processing and the "suffix_invert" field for inverted processing.

## Model Parameters

Specify the following custom model parameters if applicable:

- "dim_f": The value for the `dim_f` parameter.
- "dim_t": The value for the `dim_t` parameter.
- "n_fft": The value for the `n_fft` parameter.
- Check the box next to "use_custom_compensation" if you have your own compensation value for your model. Specify the compensation value in the "compensation" field.

## Extras

If you're working with Deezer tracks, paste your ARL (Authentication Request Library) in the "arl" field to directly access the tracks.

Specify the "Track format" by selecting the desired quality/format for the track.

To print the settings being used in the run, check the box next to "print_settings".

That's it! You're now ready to use this Colab notebook. Enjoy!

## For more detailed guide, proceed to this <a href="https://docs.google.com/document/d/17fjNvJzj8ZGSer7c7OFe_CNfUKbAxEh_OBv94ZdRG5c">link</a>.
credits: (discord) deton24